# DAML4  notes
## Week 3 - Preprocessing, PCA, clustering
<hr style="border:2px solid black"> </hr>

## Sklearn and a quick note

Sklearn (or Scikit-learn) is a high-level machine learning (ML) library for Python. It integrates with NumPy for arrays, SciPy for optimisation, and Pandas for dataframes (tabular data). It is really, really good and we will use it heavily in this course. It is high level, which lets us spend less time hacking algorithms ourselves, and more time seeing how they work.

Sklearn lets us perform PCA and K-means. "But we haven't got to the ML part of the course. What are they doing a machine learning library?" I hear you say. Well, some people do consider them to be *unsupervised machine learning* techniques, and other people don't. 



Sometimes they form part of a pipeline that eventually involves some ML. For narrative convenience, we will call them **exploratory data analysis** techniques, and leave it as that.

## Preprocessing

### Vectors and matrices

Many data analysis and machine learning techniques require us to represent our data points as vectors of real values, each of the same dimensionality $D$. In machine learning it is convention to use *column* vectors, so we will stick to this.

We can refer to some arbitrary data point as $\mathbf{x}\in \mathbb{R}^{D}$ and if we want to refer to specific data points in our dataset we can write $\mathbf{x}^{(0)},\mathbf{x}^{(1)},\mathbf{x}^{(2)},\dots ,\mathbf{x}^{(N-1)}$ where the ordering isn't important as long as it is consistent. 

$$\mathbf{x} =\begin{bmatrix}
x_0\\x_1\\\vdots \\x_{D-1} \\
\end{bmatrix}
\quad \quad
\mathbf{x}^{(n)} =
\begin{bmatrix}
x_0^{(n)}\\x_1^{(n)}\\ \vdots\\x_{D-1}^{(n)} \\
\end{bmatrix}$$




The whole dataset is the collection of these vectors $\{\mathbf{x}^{(n)}\}_{n=0}^{N-1}$ and can be stored in a dataset matrix $\mathbf{X}$. Annoyingly, it is convention to have the *rows* of this matrix representing different data points so that $\mathbf{X}\in \mathbb{R}^{N\times D}$. I tend to refer to this as a dataset matrix, but it is often referred to as a design matrix in the literature.


$$\mathbf{X} =
\begin{bmatrix}
\mathbf{x}^{(0)^\top}\\
\mathbf{x}^{(1)^\top}\\
\mathbf{x}^{(2)^\top}\\
\vdots \\
\mathbf{x}^{(N-1)^\top}\\
\end{bmatrix}
=
\begin{bmatrix}
x_0^{(0)}&x_1^{(0)}&\dots& x_{D-1}^{(0)} \\
x_0^{(1)}&x_1^{(1)}&\dots& x_{D-1}^{(1)} \\
x_0^{(2)}&x_1^{(2)}&\dots& x_{D-1}^{(2)} \\
\dots&\dots&\ddots& \vdots \\
x_0^{(N-1)}&x_1^{(N-1)}&\dots& x_{D-1}^{(N-1)} \\
\end{bmatrix}$$

Given some tabular data, we need to convert it into this $\mathbf{X}$ format. If the variables in our table are all continous variables then this is very straightforward. In fact, with a table represented as a pandas dataframe it only requires a single line of code.

Let's look at the iris dataset from the lecture. We will start by having it as a dataframe and then convert it into a dataset matrix.

In [ ]:
# Get loader for the iris dataset and import pandas and numpy
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

# Convert into dataframe with the columns as the variable names
data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)

df

To convert this into $\mathbf{X}\in \mathbb{R}^{N\times D}$ we just have to run a line of code.

In [ ]:
# Convert dataframe into array
X = np.array(df)

# Check the shape of X
print(f"X has shape {X.shape}")

### Standardisation

Standardising your data is **extremely important for PCA, K-means and many machine learning algorithms.** It allows variables to be compared on a like-for-like basis so that variables with naturally large measurements don't dominate.

For some of the toy examples in the lectures and notebooks where I have generated synthetic data, I haven't bothered with standardisation. There are two reasons for this:

1. It becomes more fiddly to make the kind of data I want to use to illustrate a point
2. The variables in these synthetic datasets are all at similar scales and things still works (they are  *roughly* standardised)

**This will rarely be the case with real data! When in doubt, standardise your data, seriously! :)**

To do this, for each variable we compute the mean and standard deviation (SD) of the measurements across all data points. For each measurement of that variable, we subtract the mean and divide by the SD.

We can do this very easily in sklearn using the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) class.

In [ ]:
# Make a copy of our dataset to compare stats after
X_old = np.array(df)

# Import scaler
from sklearn.preprocessing import StandardScaler

# Make a scaler object
scaler = StandardScaler()

# Fit the scaler to the dataset. This means computing the means and STDs
scaler.fit(X)

# Finally apply the scaler to transform the data
X = scaler.transform(X)


# Compare stats
print(
    f"Before standardisation variable means were {np.round(X_old.mean(0),2)} and SDs were {np.round(X_old.std(0),2)}"
)
print(
    f"After standardisation variable means are {np.round(X.mean(0),2)} and SDs are {np.round(X.std(0),2)}"
)

## Principal Component Analysis (PCA)

PCA takes a **standardised** dataset matrix $\mathbf{X}\in \mathbb{R}^{N\times D} $ and returns an orthornormal matrix $\mathbf{W}\in \mathbb{R}^{D\times D}$. We learnt how to compute this in the lecture. 

$\mathbf{Y}=\mathbf{XW}$ gives us a *transformed* dataset matrix $\mathbf{Y}\in \mathbb{R}^{N\times D} $ where the new dimensions $y_0,y_1,\dots,y_{D-1}$  are linear combinations of the original dimensions $x_0,x_1,\dots,x_{D-1}$. 

The significance is that there is as much variance as possible in $y_0$ so in some sense, $y_0$ best explains the data in 1D. $y_1$ is orthonormal to $y_0$ and has the next most variance and so on.

Let's revisit an example for the lecture to illustrate this. If we have a rotated ellipse, then we expect PCA to rotate it so that the major axis is along $y_0$.

First, let's make a dataset of points along a rotated ellipse.

In [ ]:
# Import preprocessing from sklearn so we can standardise in one go using X= preprocessing.scale(X)
# Also import matplotlib for plotting
from sklearn import preprocessing
import matplotlib.pyplot as plt

# This makes matplotlib output nice figures without much tweaking
plt.rcParams.update(
    {
        "lines.markersize": 10,  # Big points
        "font.size": 15,  # Larger font
        "savefig.dpi": 300,  # Higher res output
        "savefig.format": "pdf",  # PDF outputs
        "savefig.bbox": "tight",  # remove whitespace around figure
        "savefig.transparent": True,  # transparent background
        "xtick.major.size": 5.0,  # Bigger xticks
        "ytick.major.size": 5.0,  # Bigger yticks
    }
)

# Generate points on an unrotated ellipse
a = 3
b = 1
theta = np.linspace(0, 2 * np.pi, 50)
x = a * np.sin(theta)
y = b * np.cos(theta)
X = np.vstack((x, y)).T

# Rotate this by angle angle of 45 degrees anti-clockwise
t = np.pi / 4
R = np.array([[np.cos(t), np.sin(t)], [-np.sin(t), np.cos(t)]])
X = X @ R

# Standardise in a single line. This is an alternative to what we did in the last cell
X = preprocessing.scale(X)

# Finally, plot
fig, ax = plt.subplots(figsize=[6,6])
ax.set_aspect("equal", "box")
ax.grid()
ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])
ax.scatter(X[:, 0], X[:, 1])
ax.set_xlabel("$x_0$", size=20)
ax.set_ylabel("$x_1$", size=20)


Now let's use the [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) class from sklearn to compute $\mathbf{W}$ and then $\mathbf{Y}=\mathbf{XW}$. When creating an object from this class, it takes in an argument `n_components`. This refers to the number of principal components we want to keep. In this example, we want all of them so we will set this to 2.

In [ ]:
# Import PCA package
from sklearn.decomposition import PCA

# Create a pca object
pca = PCA(n_components=2)

# Calling pca.fit(X) computes W behind the scenes
pca.fit(X)

# Calling pca.transform(X) then computes X@W
Y = pca.transform(X)

# Let's now plot our transformed dataset to see if it makes sense

# Plot Y
fig, ax = plt.subplots(figsize=[6,6])
ax.set_aspect("equal", "box")
ax.grid()
ax.set_xlim([-2, 2])
ax.set_ylim([-2, 2])
ax.scatter(Y[:, 0], Y[:, 1],color='g')
ax.set_xlabel("$y_0$", size=20)
ax.set_ylabel("$y_1$", size=20)


Great! PCA can also be used for dimensionality reduction by only keeping a subset of the columns of $\mathbf{W}$. You will explore this further in the lab.

## K-means

K-means is a clustering algorithm that assigns each point to one of $K$ clusters. It should take in a **standardised** dataset, otherwise the distance computations will be skewed by the larger dimensions.

The algorithm is straightforward:

- A user decides how many clusters they want
- Cluster centres $\{\mathbf{c}_{k}\}_{k=0}^{K-1}$ are created at random 
- We then alternate between assigning points to their nearest clusters, and updating the clusters as the centre of their assigned points until there are no change

However, if you look at the [KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) class in sklearn you can see that there are several other arguments it can take in, other than just `n_clusters`. This is because the algorithms as presented to you in lectures are usually in their most elementary form. In practice there are lots of ways to tweak an algorithm to make it more efficient, faster or more appropriate for a specific scenario or dataset, but I digress.

Let's perform K-means on some synthetic data for illustration. First we will use sklearn to make a 2D dataset that consists of 4 blobs.


In [ ]:
# Sklearn lets us make synthetic datasets
from sklearn.datasets import make_blobs

# Let's make a dataset of 2D points split into 4 blobs
X, _ = make_blobs(
    n_samples=40, centers=4, n_features=2, random_state=0, cluster_std=0.4
)


fig, ax = plt.subplots(figsize=[6, 6])
ax.set_aspect("equal", "box")
ax.scatter(X[:, 0], X[:, 1], color="k")

Let's run KMeans with 4 clusters on this data, and plot the clusters in 4 different colours.

In [ ]:
# Import k-means class
from sklearn.cluster import KMeans

# Create k-means object
kmeans = KMeans(n_clusters=4, random_state=0)

# Run the k-means algorithm
kmeans.fit(X)

# We can look at kmeans.labels_ to see the assignments
print(kmeans.labels_)

# Make a colourmap
colormap = np.array(["r", "g", "b", "y"])


fig, ax = plt.subplots(figsize=[6, 6])
ax.set_aspect("equal", "box")
ax.scatter(X[:, 0], X[:, 1], c=colormap[kmeans.labels_])

Nice! Although this algorithm is very sensible to the intial cluster placement. Also, clusters don't necessarily correspond to anything meaningful!

<hr style="border:2px solid black"> </hr>

#### Written by Elliot J. Crowley and &copy; The University of Edinburgh 2022-23